# TensorFlow Playground
Welcome to the TensorFlow Playground! This notebook is modeled after the official [playground.tensorflow.org](http://playground.tensorflow.org) web application which offers an playful way to test different feed forward neural network configurations. This notebook gives you the opportunity to actually implement different network configurations you might already have tested in the web app.

## Preperations
You can basically ignore this first code cell below. All it does is generate the four different datasets and provide a visualization function. What you can do here is adjust N, the amount of samples we generate. If you feel fancy you can also tweak the datasets to make the problems harder by for example adding some noise to the spiral.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

import numpy as np
from numpy.random import normal, random_sample, permutation
from numpy.linalg import norm

def viz(data, labels, progress, accuracy):
    """Function visualizing training progress."""
    if not hasattr(fig, 'viz'): viz.fig = plt.figure('Result')
    viz.fig.suptitle("{:3.2%} done with accuracy {:2.2f}"
                     .format((epoch+1)/epochs, accuracy))
    plt.scatter(*zip(*data), c=labels, cmap='bwr')
    viz.fig.canvas.draw()


class Datasets:

    # Returns n floats uniformly distributed between a and b.
    uniform = lambda self, a, b, n=1: (b - a) * random_sample((1, n)) + a
    
    def splitsets(self, data, targets, split=10):
        """Splits samples into train and test sets and reshapes target labels as required."""
        n = data.shape[0]
        permut = permutation(n).astype(int)
        targets = np.array([targets, np.abs(targets-1)]).T
        return tuple(zip(*((d[permut[n//split:]], d[permut[:n//split]]) for d in (data, targets))))

    def circle(self, n=1000, radius=5, split=10):
        """Generates data with one class describing an inner and one an outer circle."""
        radius = 5
        r = np.append(self.uniform(0, radius*.5, n//2), self.uniform(radius*.7, radius, n//2))
        angle = self.uniform(0, 2*np.pi, n)
        xy = np.vstack((r * np.sin(angle), r * np.cos(angle)))
        t = np.less(norm(xy, axis=0), radius*.5)
        return self.splitsets(xy.T, t, split)
    
    def xor(self, n=1000, padding=.3, split=10):
        """Generates xor data."""
        x = self.uniform(-5, 5, N); x[x>padding] += padding; x[x<padding] -= padding
        y = self.uniform(-5, 5, N); y[y>padding] += padding; y[y<padding] -= padding
        t = np.less(x*y, 0).flatten()
        return splitsets(np.vstack((x, y)).T, t, split)

    def gauss(self, n=1000, split=10):
        """Generates two blops of gauss data, each describing a class."""
        return splitsets(np.vstack((normal(2, 1, (N//2, 2)), normal(-2, 1, (N//2, 2)))),
                         np.append(np.ones(N//2), np.zeros(N//2)))

    def genSpiral(self, delta, n):
        """Generates a spiral of n points with rotation-offset delta."""
        points = np.arange(n)
        r = points / n * 5
        t = 1.75 * points / n * 2 * np.pi + delta
        return np.vstack((r * np.sin(t), r * np.cos(t))).T
    
    def spiral(self, n=1000):
        """Generates a dataset of two intertwined spirals."""
        return splitsets(np.vstack((self.genSpiral(0, n//2), genSpiral(np.pi, n//2))),
                         np.append(np.ones(n//2), np.zeros(n//2)))

data = Datasets()
circle = data.circle()
xor = data.xor()
gauss = data.gauss()
spiral = data.spiral()
    
# visualize
fig = plt.figure('Available Datasets')
fig.suptitle('only showing test set')
ax1 = plt.subplot(221); ax1.set_title('circle')
ax1.scatter(*zip(*circle[1][0]), c=circle[1][1][:,0], cmap='bwr')
ax2 = plt.subplot(222); ax2.set_title('xor')
ax2.scatter(*zip(*xor[1][0]), c=xor[1][1][:,0], cmap='bwr')
ax3 = plt.subplot(223); ax3.set_title('gauss')
ax3.scatter(*zip(*gauss[1][0]), c=gauss[1][1][:,0], cmap='bwr')
ax4 = plt.subplot(224); ax4.set_title('spiral')
ax4.scatter(*zip(*spiral[1][0]), c=spiral[1][1][:,0], cmap='bwr')
fig.canvas.draw()

## Data
Select your dataset here by simply replacing the variable name at the very left. Some information about the provided data structures:

The two `*_data` variables contain the actual data in 2 dimensional numpy arrays, the first axis describes individual observations, the second axis the `x` and `y` values. Therefore they have a shape of `(m, 2)`. Similarilly the two `*_target` variables are arrays containing the corresponding labels, again in a shape of `(m, 2`), $samples \times classes$: The second axis binarily describes weather a sample belongs to the first class (the first value in the second axis will equal `1`, the second value `0`) or to the second class (second value will be `1`..).

As the names suggest, `train_data` and `train_targets` are designated for training and contain a lot more data (by default $9/10$ of the data is training data) and `test_data` and `test_targets` are meant for evaluating the networks performance.

In [ ]:
# circle, xor, gauss, spiral
(train_data, train_targets), (test_data, test_targets) = gauss

## It's your turn!
In the next cell stuff gets interesting. We preimplemented a basic model which is sufficiently powerful for the gauss dataset. It consists of 2 neurons in the input layer to feed `x` and `y` values into, 4 neurons in the single hidden layer and again 2 neurons in the output layer for giving probabilities for each of our two classes.

Instead of using buttons and dropdowns you will need to get your hands dirty here in order to being able to cope with more complex datasets! While we provide some guidance to mock the experience of the web application it will be also super useful to checkout the [TensorFlow API documentation](https://www.tensorflow.org/versions/r0.9/api_docs/index.html). Because the documentation does not give any theoretical background on all those different functions you might also want to regularily consultate Wikipedia.

  * Adjust the hidden layer, play around with more or even fewer neurons.
  * Add another hidden layer! You will need to adjust the matrix multiplications for its surrounding layers in order to integrate it.
  * Change the activation functions. By default both, the hidden and the output layer, use the sigmoid activation function. Similar to the web application you can use `relu` or `tanh`. You might also consider trying to implement your own `linear` activation function -- TensorFlow does not provide one by itself.
  * Take a look into different error measures and optimizers. This is nothing you can change in the playground web application but actually a powerful tool to tweak your networks performance through better training. Checkout the [optimizers chapter](https://www.tensorflow.org/versions/r0.9/api_docs/python/train.html#optimizers) in the documentation.
  * Lastly, take a look at the actual training loop. Tweak the amount of iterations and change the batch size. You can actually also tweak the batch size in the web app, take a look at the bottom left. For changing the ratio of training to test data and to add noise to the data you will need to hack around in the top code cell.

In [ ]:
import tensorflow as tf

x = tf.placeholder(tf.float32, shape=[None, 2], name='input_layer')

W_h = tf.Variable(tf.random_normal([2,4]), name='hidden_weights')
b_h = tf.Variable(tf.zeros([4]), name='hidden_bias')
h = tf.nn.sigmoid(tf.matmul(x,W_h) + b_h, name='hidden_layer')

W_y = tf.Variable(tf.random_normal([4,2]), name='output_weights')
b_y = tf.Variable(tf.zeros([2]), name='output_bias')
y = tf.nn.sigmoid(tf.matmul(h,W_y) + b_y, name='output_layer')

t = tf.placeholder(tf.float32, shape=[None, 2], name='targets')

error = tf.reduce_mean(tf.squared_difference(t,y), name='mean_squared_error')
optimizer = tf.train.GradientDescentOptimizer(0.03).minimize(error, name='gradient_descent')

prediction_check = tf.equal(tf.argmax(y,1), tf.argmax(t,1), name='prediction_check')
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32), name='accuracy_measure')

sess = tf.Session()
sess.run(tf.initialize_all_variables())

epochs = 1000
for epoch in range(epochs):
    split = permutation(len(train_targets))
    batch_targets = train_targets[split[:10]]
    batch_data = train_data[split[:10]]
    sess.run(optimizer, {x: batch_data, t: batch_targets})
    if (epoch+1)%100 == 0:
        perf = sess.run(accuracy, {x: test_data, t: test_targets})
        predictions = sess.run(y, {x: test_data})
        viz(test_data, predictions[:,0], (epoch+1)/epochs, perf)

perf = sess.run(accuracy, {x: test_data, t: test_targets})
predictions = sess.run(y, {x: test_data})
viz(test_data, predictions[:,0], 1, perf)